# Delta-Neutral Simulation 

**TODO: Lots of documentation around what this does, all the parameters, etc.**

**Setup**
1. Navigate to the [Configuration & Scenario Setup](#configuration_setup) section.
1. Update base parameters.
1. Define reward strategy as one of the functions in the [Reward Strategy Definition](#reward_strategy_definition) section.
1. Initialize a trade strategy definition in the [Trade Strategy Definition](#trade_strategy_definition) section.
1. Define a Price Movement Scenario using a function in the [Price Movement Scenario](#price_movement_scenario) section, or create your own.
1. Run
1. View table, text, and plot outputs at the bottom.

**References**: 
* The Best Resource. Used for Benchmarking, Unit Testing, and Key formulas:
[Alpaca Finance Yield Farming Calculator](https://docs.google.com/spreadsheets/d/15pHFfo_Pe66VD59bTP2wsSAgK-DNE_Xic8HEIUY32uQ/edit#gid=650365.25877)
  * 1.1) Pseudo delta-neutral LP farming
  * 3.2) Double-sided Farming
* Impermanent loss equations: [Uniswap Understanding Returns](https://docs.uniswap.org/protocol/V2/concepts/advanced-topics/understanding-returns)
* Details on LPs/DeFi: [DeFi Guide for Newbie (and how to manage risk)](https://www.reddit.com/r/defi/comments/rxj072/defi_guide_for_newbie_and_how_to_manage_risk/)
* [Do fees earned from liquidity pools behave like compound interest?](https://www.reddit.com/r/UniSwap/comments/ldmq18/do_fees_earned_from_liquidity_pools_behave_like/)
  * tl;dr: Every transaction effectively acts as a compounding event. So low-activity pools have lower compounding than high-activity pools. But can be generally treated as compounding. 
* Debt Ratio: [Francium Liquidation Docs](https://docs.francium.io/product/liquidation)
* Overview of different ways to open a Pseudo Delta Neutral position [Revisiting the Fundamentals of Pseudo-Delta Neutral Hedging - DarkRay](https://darkray.medium.com/revisiting-the-fundamentals-of-pseudo-delta-neutral-hedging-4da279caabfa)

**Notes**:
* `token0` always assumed to be the stable, and `token1` is the (risky) asset that is borrowed at a higher rate

**Equations**:

Rough equation to calculate leveraged Yield Farming returns without factoring in compounding, rewards, fees, etc:
```
E = L*P*(1 + r_e/n)^(n*t) - (L-1)*P*(1 + r_b/n)^(n*t)
```
where:
```
E: final equity
L: Leverage amount (1, or >2)
P: Initial principal
r_e: earning interest rate (such that 0.5=50%)
r_b: borrow interest rate (such that 0.5=50%)
n: number of times compounded a year, can assume daily (365.25).
t: number of years
```
This can be used as a comparison to this sim if you set all rates/fees to zero except trading fees, and no variation.

# TODOs
---
- [ ] Spot check rebalance -- check the IL losses. Should have some -- Compare to no-op. Compare to a no-price-change scenario. i.e. is it doing better than if there's no price change at same APRs? If so, can't be possible I think -- See notes in evernote. Something fishy?
- [ ] Fix impermanent loss calculations. Should reset on every rebalance
- [ ] Commit clean
- [ ] Add colab button
- [ ] Set up a downloader on the file (github)
- [ ] Document description

# Install Packages
---

In [ ]:
# !pip install -r requirements.txt

# Imports
---

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Import all the support strategies & scenarios so a user of this notebook can select any of them.
import scenarios
import simulation
import strategies.rewards as reward_strategies
import strategies.trading as trading_strategies

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

<a id='configuration_setup'></a>
# Configuration & Scenario Setup
---

## Parameters

In [ ]:
"""
Configuration that is always used, regardless of scenario
"""
# Initial amount invested
initial_cap = 10000

# Swap fee, note: 0.01 = 1%
fee_swap = 0.00

# Cost of gas for each transaction, in dollars
fee_gas = 0

# Open a leveraged position
# Options: [1.0, 2.0, >2.0]
leverage = 3

# Price of reward token
reward_token_price = 1.0

# Liquidation threshold
# Note: Francium threshold is 83.3%. See Francium reference at top.
# Currently unused. For plotting purposes only.
liquidation_threshold = 0.833

"""
Configuration that is used if the scenario does not internally generate the data itself
"""
# Earnings APYs, note: 1.0 = 100%
apy_trading_fee = 0.3
apy_reward = 0.0

# Borrow / Lending APYs
apy_borrow_token0 = 0.1
apy_borrow_token1 = 0.1


In [ ]:
"""
Configuration that is used if `create_history_from_files()` is selected as the scenario. 
That generator derives all pool APYs, borrow APYs, and prices
"""
# Token names, according to CREAM, to lookup borrow rates. 
# May differ from CoinDix pool symbol name
cream_token0_name = "USDC.e" 
cream_token1_name = "WAVAX"

# CoinGecko config
token1_coingecko_id = "avalanche-2"
reward_token_coingecko_id= "joe"

# CoinDix parameters
coindix_pair = "USDC.e-AVAX"
protocol = "trader joe"
chain = "avalanche"

<a id='reward_strategy_definition'></a>
## Reward Strategy Definition
Define a reward function using one found in [Reward Token Strategies](#reward_token_strategies) section. Or create your own and reference it here.

In [ ]:
# Define parameters for what to do with rewards
# To HODL, use any SellRewardsStrategy(sell_amount=0)
rewards_sell_amount = 1.0

# Options: SellRewardsStrategy, CompoundRewardsStrategy
rewards_cls = reward_strategies.SellRewardsStrategy(sell_amount=rewards_sell_amount)
# rewards_cls = reward_strategies.CompoundRewardsStrategy(sell_amount=rewards_sell_amount)

<a id='price_movement_scenario'></a>
## Price Movement Scenario

Uncomment the scenario to try it. Or create your own from a function in the [Scenario Generators](#scenario_generators) section.

In [ ]:
"""
Uncomment to create a small 4-day example that unit-tests: no change, very negative price change, very positive price change
"""
# pp = scenarios.create_small_example()


"""
Uncomment this to run for one year without any price change -- useful to test APY growths and fees without price impact
"""
# pp = scenarios.create_no_price_change_example(365)


"""
Uncomment this to run for one year, decaying price change to 80% losses -- useful to see dramatic effects on investment
"""
# pp = scenarios.create_linear_price_change_example(365, token0_price=1.0, token1_price_initial=0.1, token1_price_final=0.02)


"""
Uncomment this to run for one year, increasing price 5X -- useful to see dramatic effects on investment
"""
# pp = scenarios.create_linear_price_change_example(365, token0_price=1.0, token1_price_initial=0.1, token1_price_final=0.5)


"""
Uncomment this to run for one year, without any non-safe-asset (NSA) price change, but the reward token drains to near zero.
This is an example of typical farming token where it may be better to sell every day instead of compound or hold, 
depending on initial capital and gas fees.
"""
# pp = scenarios.create_linear_price_change_example(
#     365, token0_price=1.0, token1_price_initial=0.1, token1_price_final=0.1, reward_token_price_initial=1.0, reward_token_price_final=0.01
# )

"""
Uncomment this to go from price of 1.0 down to 0.2 and back to 0.1 within a year. Tests to see the effect of returning to no IL 
when price dumps and comes back
"""
# pp = scenarios.create_linear_and_back_example(365, token0_price=1.0, token1_price_base=0.1, token1_price_peak=0.01)

"""
Uncomment this to go from price of 1.0 down to 5.0 (5X) and back to 0.1 within a year. Tests to see the effect of returning to 
no IL when price pumps and comes back
"""
# pp = scenarios.create_linear_and_back_example(365, token0_price=1.0, token1_price_base=0.1, token1_price_peak=0.3)

"""
Uncomment this to take a random walk. 
Tips: 
- Set seed to None for random every time, or integer to be deterministic
- Set bias and variance based in terms of daily-percent movement. 
- See defaults for reasonable starts. Try plotting first as well.
- Set bias to positive to walk upward, negative to walk downward, or 0 to randomly move stationary. 
- Set variance to define how much the walk should move each step.
"""
pp = scenarios.create_random_walk_example(365, token1_price_initial=10, bias=-0.002, variance=0.005, seed=1234)

"""
Uncomment this to run a real-world example using actual historical data.
Reads in AVAX-USDC.e pool example from CoinDix/CREAM and prices from coingecko
Assumes 10% of total APY is rewards (coindix doesn't report reward APY for this pool, so just an assumption)
"""
# pp = scenarios.create_history_from_files(
#     coindix_pair,
#     protocol,
#     chain,
#     cream_token0_name, 
#     cream_token1_name,
#     token1_coingecko_id,
#     reward_token_coingecko_id,
#     reward_apy_ratio=0.10
# )

# Suppress auto-comment print above
print("")

<a id='trade_strategy_definition'></a>
## Trade Strategy Definition
Initialize a trade strategy object found in [Trade Strategies](#trade_strategies)

In [ ]:
# Flag to open-position on init. Should be set to False for strategies that decide when to open themselves
open_on_start = True

"""
This strategy is a no-op. It takes no action and is just a HODL.
"""
# strategy_cls = trading_strategies.HODLStrategy()

"""
This strategy rebalances whenever the price moves 5% from the last rebalance price. It does this by just closing and re-opening
the position. In reality, this would be done through collateral rebalancing but the net effect is the same. To accommodate for 
this difference, gas and swap fees are reduced here.
"""
strategy_cls = trading_strategies.RebalanceStrategy(
        pp["token1_price"][0], 
        0.3,
        leverage,
        1.0,
        fee_swap=fee_swap/10.0,  # In real-world, the swap will be a balancing of a small percentage of the total position
        fee_gas=fee_gas/3.0
)

# Suppress odd comment prints
print("")

# Initialize
---

# Run Simulation
---

In [ ]:
pp = simulation.simulate(
    pp,
    strategy_cls,
    rewards_cls,
    initial_cap,
    leverage,
    fee_gas,
    fee_swap,
    reward_token_price,
    apy_trading_fee,
    apy_reward,
    apy_borrow_token0,
    apy_borrow_token1,
    open_on_start
)

# Table Summary
---

In [ ]:
pp.T

# Text Summary
---

In [ ]:
days_elapsed = (pp.index[-1] - pp.index[0]).days + 1
final_row = pp.iloc[-1]
print(f"Initial Wallet Value: ${initial_cap:0.2f}")
print(f"Days Elapsed: {days_elapsed}")
print(f"Number of Trade Strategy Executions: {pp['trade_event'].sum()}")
print(f"Final Wallet Value: ${final_row['equity_value']:0.2f}")
print(f"Final Profit: ${final_row['profit_value']:0.2f}")
print(f"Effective APR: {final_row['annualized_apr'] * 100:0.2f}%")

# Plot Results
---

In [ ]:
# Filter trade events and anchor to top of plot
tradey = pp["token1_price"].max()
trade_points_plot = pp["trade_event"] * tradey
trade_points_plot = trade_points_plot[trade_points_plot>0]

fig = go.Figure()
fig.add_trace(go.Scatter(x=pp.index, y=pp["token1_price"], mode='lines', name='token1_price'))
fig.add_trace(go.Scatter(
    x=trade_points_plot.index, y=trade_points_plot, mode='markers', name='Trade Events', marker_line_width=2, marker_size=7
))
fig.update_layout(
    title="Price of token1 over time, and when Trade Strategy events executed",
    xaxis_title="Price ($)",
    yaxis_title="date"
)


In [ ]:
# px.line(pp, y="reward_token_price", title=f"Reward Token Price Over Time, final=${pp['reward_token_price'][-1]:.2f}")

In [ ]:
print("Note: Total Equity = Pool Equity + Cash + Unrealized Rewards - Fees")
px.line(pp, y=["equity_value"], title=f"Total Equity Value Over Time, final=${pp['equity_value'][-1]:.2f}")

In [ ]:
# px.line(pp, y=["token0_supply_close", "token1_supply_close", "token0_debt_close", "token1_debt_close",], title=f"Token Supply and Debt Over Time")

In [ ]:
# px.line(pp, y=["debt_value", "position_value"], title="Position and Debt Value Over Time")

In [ ]:
# fig = px.line(pp, y=["effective_leverage"], title=f"Effective Leverage Over Time (Pool Position / Pool Equity), final={pp['effective_leverage'][-1]:.2f}")
# fig.update_yaxes(range=[0, 3.5], autorange=True)

In [ ]:
# px.line(pp, y=["profit_value"], title=f"Profit Over Time, final=${pp['profit_value'][-1]:.2f}")

In [ ]:
fig = px.line(pp, y="roi", title=f"ROI % On a Given Date, final={pp['roi'][-1]*100:.2f}%")
fig.layout.yaxis.tickformat = ',.0%'
fig.show()

In [ ]:
# px.line(pp, y=pp["annualized_apr"]*100, title=f"Annualized APR At Point in Time, final={pp['annualized_apr'][-1]*100:.2f}%")

In [ ]:
# fig = px.line(pp, y=["apy_trading_fee", "apy_reward"], title="Accrual APYs Over Time")
# fig.layout.yaxis.tickformat = ',.0%'
# fig.show()

In [ ]:
# fig = px.line(pp, y=["apy_borrow_token0", "apy_borrow_token1"], title=f"Borrow APYs Over Time")
# fig.layout.yaxis.tickformat = ',.0%'
# fig.show()

In [ ]:
# fig = px.line(pp, y="iloss", title=f"Impermanent Loss Over Time (price change only, unrelated to position value), final={pp['iloss'][-1]*100:.2f}%")
# fig.layout.yaxis.tickformat = ',.0%'
# fig.show()

In [ ]:
# px.line(pp, y=["rewards_value", "cash_value"], title=f"Rewards Value and Cash Value Accumulated, finals = [${pp['rewards_value'][-1]:.2f}, {pp['cash_value'][-1]:.2f}]")

In [ ]:
# px.line(pp, y=["fees_value"], title=f"Gas Fees Accumulated, final = ${pp['fees_value'][-1]:.2f}")

In [ ]:
# print("Debt Ratio here is defined as `debt_value / (debt_value + pool_equity)`. Since it's used to consider liquidation, it is a function of the LP alone, and not fees/rewards/cash.")
# fig = px.line(pp, y=["debt_ratio"], title=f"Debt Ratio over Time, final = {pp['debt_ratio'][-1]:3f}")
# fig.add_hline(y=liquidation_threshold, line_width=3, line_dash="dash", line_color="red")
# fig.update_yaxes(range=[0, 1], autorange=False)
# fig.show()

In [ ]:
px.line(pp, y=["position_hodl_dollars"], title=f"Value if HODL'd original position (50% volatile asset, 50% stable), final = ${pp['position_hodl_dollars'][-1]:.2f}")


In [ ]:
# Unpack metadata
df_metadata = pd.DataFrame([zz for zz in pp["strategy_metadata"]], index=pp.index)
if df_metadata.shape[1] > 0:
    fig = px.line(df_metadata, title=f"Metadata Output from Strategy Over Time")
    fig.show()
else:
    print("No strategy metadata found. Skipping plot.")

# End